# 音象徴の機械学習による再現：最強のポケモンの生成

#### 三浦 智 ∗1 村田 真樹 ∗1 保田 祥 ∗2 宮部 真衣 ∗2 荒牧 英治 ∗2
##### ∗1 鳥取大学大学院  *2 東京大学   言語処理学会 第18 回年次大会 発表論文集 (2012年  3月)　

<br>

## 目次

- NLP(自然言語処理)でデータを扱うためには

- tokenize
 
 これこれ
- Embedding


### NLPでデータを扱うためには

**機械に文字を認識させることはできない**

文字 →→→ 数字

<br>

上の処理を行うためには, 大きく二つの処理が必要.

*tokenize* & *Embedding*

### tokenizeとは

基本的には文章形を扱うことが多い自然言語処理, そこで必要なのが文章を**分割**すること.  

Ex : 本山は、足利義満により建立された京都の相国寺。

<br>

|分割手法|分割|分割数|
|:-|:-|:-|
|SentencePiece (8k)|本 山 は 、 足利義 満 により 建立 された 京都 の 相 国 寺 。|15|
|KyTea|本山 は 、 足利 義満 に よ り 建立 さ れ た 京都 の 相国 寺 。|17|
|MeCab|本山 は 、 足利 義満 により 建立 さ れ た 京都 の 相国寺 。|14|
|MeCab-neologd|本山 は 、 足利義満 により 建立 さ れ た 京都 の 相国寺 。|12|

<br>

英語なら元々分割されてるが, 日本語ではそうはいかない.
~~日本語ほんと使い辛い~~

### Embeddingとは

分割した塊ごとに数字に置き換えること.  

Ex : 本 山 は 、 足利義 満 により 建立 された 京都 の 相 国 寺 。  
Embeded : 12, 105, 22, 1, 4, 6, 65, 33, 20, 5, 32, 2, 76, 44, 34


#### Embeddingの手法紹介
- One-hot
- Word2vec
- fastText

##### One-hot
一番シンプルなEmbedding手法.    
まず表現したい語彙リストを作成し, 各単語を表現する次元を準備する.  
表現したい文に含まれている単語に対応する次元を１に, それ以外を0にする方法.  
文のベクトルを作るBag-of-words（BoW）表現に利用される.

##### Word2vec
大量のテキストデータを解析し, 各単語の意味をベクトル表現化する手法.    
単語をベクトル化することで, 単語同士の意味の近さを計算した, 単語同士の意味を足したり引いたりすることが可能.  

##### fastText
これまでの手法は単語をベースとするため、未知語に対応するのが難しい.   
これを克服するためのアイデアとして、単語より小さな単位でEmbedding を行う.   
文字レベルのN-gram (Character N-gram) であるsub-wordを用いる.   
*ただし日本語に適応させることは難しい

# 音象徴の機械学習による再現：最強のポケモンの生成

上記を踏まえると...    
ポケモンの名前を数字に変換する必要がある.

紹介したEmbedding手法では適応できない.    
論文にも詳細は書かれていない.....

### 今回取る手法

N-gramをベースに一文字ずつ数字に置き換える.
(多分、論文もこの手法な気がする)

Ex : フシギダネ → [23, 12, 52, 61, 24]

ポケモンの名前はMAX6文字なので, 字余りの時は適当なトークンをつけて変換.

Ex : フシギダネ → [23, 12, 52, 61, 24, 0]   
Ex : ヒトカゲ→ [32, 20, 6, 54, 0, 0]

*LSTM等の深層学習における学習データは大体この形に変換されて使われてる.

今回使用したデータ(第七世代まで）
>https://rikapoke.hatenablog.jp/entry/pokemon_datasheet_gne7

In [1]:
#OMAJINAI
import warnings
warnings.filterwarnings('ignore')

In [2]:
# データを見ていく
import pandas as pd
status = pd.read_csv("pokemon_status.csv", encoding="shift_jis")
status

,図鑑番号,ポケモン名,タイプ１,タイプ２,通常特性１,通常特性２,夢特性,HP,こうげき,ぼうぎょ,とくこう,とくぼう,すばやさ,合計
0,1,フシギダネ,くさ,どく,しんりょく,NaN,ようりょくそ,45,49,49,65,65,45,318.0
1,2,フシギソウ,くさ,どく,しんりょく,NaN,ようりょくそ,60,62,63,80,80,60,405.0
2,3,フシギバナ,くさ,どく,しんりょく,NaN,ようりょくそ,80,82,83,100,100,80,525.0
3,003-1,メガフシギバナ,くさ,どく,あついしぼう,NaN,NaN,80,100,123,122,120,80,625.0
4,4,ヒトカゲ,ほのお,NaN,もうか,NaN,サンパワー,39,52,43,60,50,65,309.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
904,798,カミツルギ,くさ,はがね,ビーストブースト,NaN,NaN,59,181,131,59,31,109,570.0
905,799,アクジキング,あく,ドラゴン,ビーストブースト,NaN,NaN,223,101,53,97,53,43,570.0
906,800,ネクロズマ,エスパー,NaN,プリズムアーマー,NaN,NaN,97,107,101,127,89,79,600.0
907,801,マギアナ,はがね,フェアリー,ソウルハート,NaN,NaN,80,95,115,130,115,65,600.0


### データの前処理編
余分なデータを削除したい.    
今回使いたいデータは"ポケモンの名前", "合計(種族値)".   
余計なデータの削除 & メガポケモンのデータを削除.

In [3]:
#余計なメガポケモンの削除
status = status[~status['図鑑番号'].str.contains('-')]
status

,図鑑番号,ポケモン名,タイプ１,タイプ２,通常特性１,通常特性２,夢特性,HP,こうげき,ぼうぎょ,とくこう,とくぼう,すばやさ,合計
0,1,フシギダネ,くさ,どく,しんりょく,NaN,ようりょくそ,45,49,49,65,65,45,318.0
1,2,フシギソウ,くさ,どく,しんりょく,NaN,ようりょくそ,60,62,63,80,80,60,405.0
2,3,フシギバナ,くさ,どく,しんりょく,NaN,ようりょくそ,80,82,83,100,100,80,525.0
4,4,ヒトカゲ,ほのお,NaN,もうか,NaN,サンパワー,39,52,43,60,50,65,309.0
5,5,リザード,ほのお,NaN,もうか,NaN,サンパワー,58,64,58,80,65,80,405.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
904,798,カミツルギ,くさ,はがね,ビーストブースト,NaN,NaN,59,181,131,59,31,109,570.0
905,799,アクジキング,あく,ドラゴン,ビーストブースト,NaN,NaN,223,101,53,97,53,43,570.0
906,800,ネクロズマ,エスパー,NaN,プリズムアーマー,NaN,NaN,97,107,101,127,89,79,600.0
907,801,マギアナ,はがね,フェアリー,ソウルハート,NaN,NaN,80,95,115,130,115,65,600.0


In [4]:
status['len'] = status['ポケモン名'].map(lambda x: len(x))
status

,図鑑番号,ポケモン名,タイプ１,タイプ２,通常特性１,通常特性２,夢特性,HP,こうげき,ぼうぎょ,とくこう,とくぼう,すばやさ,合計,len
0,1,フシギダネ,くさ,どく,しんりょく,NaN,ようりょくそ,45,49,49,65,65,45,318.0,5
1,2,フシギソウ,くさ,どく,しんりょく,NaN,ようりょくそ,60,62,63,80,80,60,405.0,5
2,3,フシギバナ,くさ,どく,しんりょく,NaN,ようりょくそ,80,82,83,100,100,80,525.0,5
4,4,ヒトカゲ,ほのお,NaN,もうか,NaN,サンパワー,39,52,43,60,50,65,309.0,4
5,5,リザード,ほのお,NaN,もうか,NaN,サンパワー,58,64,58,80,65,80,405.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
904,798,カミツルギ,くさ,はがね,ビーストブースト,NaN,NaN,59,181,131,59,31,109,570.0,5
905,799,アクジキング,あく,ドラゴン,ビーストブースト,NaN,NaN,223,101,53,97,53,43,570.0,6
906,800,ネクロズマ,エスパー,NaN,プリズムアーマー,NaN,NaN,97,107,101,127,89,79,600.0,5
907,801,マギアナ,はがね,フェアリー,ソウルハート,NaN,NaN,80,95,115,130,115,65,600.0,4


In [5]:
de = status[status['len']>6]
status = status[status['len']<7]
de

,図鑑番号,ポケモン名,タイプ１,タイプ２,通常特性１,通常特性２,夢特性,HP,こうげき,ぼうぎょ,とくこう,とくぼう,すばやさ,合計,len
721,641,トルネロス化身,ひこう,NaN,いたずらごころ,NaN,まけんき,79,115,70,125,80,111,580.0,7
723,642,ボルトロス化身,でんき,ひこう,いたずらごころ,NaN,まけんき,79,115,70,125,80,111,580.0,7
727,645,ランドロス化身,じめん,ひこう,すなのちから,NaN,ちからずく,89,125,90,115,80,101,600.0,7
766,678,ニャオニクス♂,エスパー,NaN,するどいめ,すりぬけ,いたずらごころ,74,48,76,83,81,104,466.0,7
803,710,バケッチャ特大,ゴースト,くさ,ものひろい,おみとおし,ふみん,59,66,70,44,55,41,335.0,7
807,711,パンプジン特大,ゴースト,くさ,ものひろい,おみとおし,ふみん,85,100,122,58,75,54,494.0,7
814,718,ジガルデ10%,ドラゴン,じめん,オーラブレイク,スワームチェンジ,54,100,71,61,85,115,486,NaN,7
819,720,フーパ(戒め),エスパー,ゴースト,マジシャン,NaN,NaN,80,110,60,150,130,70,600.0,7
841,741,オドリドリ \n (めらめら),ほのお,ひこう,おどりこ,NaN,NaN,75,70,70,98,70,93,476.0,14
879,774,メテノ(流星),いわ,ひこう,リミットシールド,NaN,NaN,60,60,100,60,100,60,440.0,7


In [6]:
status = status.loc[:, ['ポケモン名','合計']]
status

,ポケモン名,合計
0,フシギダネ,318.0
1,フシギソウ,405.0
2,フシギバナ,525.0
4,ヒトカゲ,309.0
5,リザード,405.0
...,...,...
904,カミツルギ,570.0
905,アクジキング,570.0
906,ネクロズマ,600.0
907,マギアナ,600.0


In [7]:
#tokenize
def function(name):
    n_gram = ''
    for n in name:
        n_gram = n_gram + n + ' '
    return n_gram

status['ポケモン名'] = status['ポケモン名'].map(function)
status

,ポケモン名,合計
0,フ シ ギ ダ ネ,318.0
1,フ シ ギ ソ ウ,405.0
2,フ シ ギ バ ナ,525.0
4,ヒ ト カ ゲ,309.0
5,リ ザ ー ド,405.0
...,...,...
904,カ ミ ツ ル ギ,570.0
905,ア ク ジ キ ン グ,570.0
906,ネ ク ロ ズ マ,600.0
907,マ ギ ア ナ,600.0


In [8]:
#種族値の正規化
from sklearn import preprocessing
status['合計'] = preprocessing.minmax_scale(status['合計'])
status

,ポケモン名,合計
0,フ シ ギ ダ ネ,0.262385
1,フ シ ギ ソ ウ,0.422018
2,フ シ ギ バ ナ,0.642202
4,ヒ ト カ ゲ,0.245872
5,リ ザ ー ド,0.422018
...,...,...
904,カ ミ ツ ル ギ,0.724771
905,ア ク ジ キ ン グ,0.724771
906,ネ ク ロ ズ マ,0.779817
907,マ ギ ア ナ,0.779817


In [9]:
def labeling(pred, p=0.5):
    if pred < p:
        pred_label = 0
    else:
        pred_label = 1
    
    return pred_label

status['合計'] = status['合計'].map(labeling)
status

,ポケモン名,合計
0,フ シ ギ ダ ネ,0
1,フ シ ギ ソ ウ,0
2,フ シ ギ バ ナ,1
4,ヒ ト カ ゲ,0
5,リ ザ ー ド,0
...,...,...
904,カ ミ ツ ル ギ,1
905,ア ク ジ キ ン グ,1
906,ネ ク ロ ズ マ,1
907,マ ギ ア ナ,1


### 学習編
ここまでで, 学習に必要な形にデータの整形が行えた.   
次に, このデータを用いて, LSTMを回していく.　　　

####  学習の目的
ポケモンの名前から, 合計種族値を推定.
データを”学習データ”, "検証用データ"に分けたのち, 精度推定.  
論文と同じ手法で**最強のポケモン**の生成を行う.



###### 若干の言い訳
本来ならば論文通り, SVMで学習を進めていこうと考えていたが, 手法が古すぎてpythonの便利モジュールに欲しいような物がない.   
書けなくないけどこういう無駄な作業は嫌いなので, 深層学習で同じようなことをやる.



In [10]:
from torchtext import data
import torchtext

In [11]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(status, random_state=1234, test_size=0.2)
train_df.to_csv("./train_df.tsv", sep='\t')
val_df.to_csv("./val_df.tsv", sep='\t')

In [12]:
batch_size = 4
max_len = 6

# tokenizeの方法
tokenizer = lambda x: x.split()

# ラベル情報等
TEXT = data.Field(sequential=True, tokenize=tokenizer, include_lengths=True, 
                 batch_first=True, fix_length=max_len)
LABEL = data.LabelField()

fields_train = [('id', None), ('name', TEXT), ('bs', LABEL)]

dataset_train, dataset_valid = data.TabularDataset.splits(
                             path = './',
                             format='TSV',
                             skip_header=True, 
                             train="train_df.tsv",
                             validation="val_df.tsv", 
                             fields=fields_train)

In [13]:
#中身のチェック
vars(dataset_train[0])

{'name': ['ハ', 'ク', 'リ', 'ュ', 'ー'], 'bs': '0'}

In [14]:
TEXT.build_vocab(dataset_train)
LABEL.build_vocab(dataset_train)

train_iter = data.BucketIterator(dataset=dataset_train, batch_size=batch_size, 
                                 sort_key=lambda x: len(x.name), repeat=False, shuffle=True)
val_iter = data.BucketIterator(dataset=dataset_valid, batch_size=1, 
                                 sort_key=lambda x: len(x.name), repeat=False, shuffle=False)

In [15]:
# Embeddingのサンプル確認
batch = next(iter(val_iter))
print(batch.name)
print(TEXT.vocab.itos)

(tensor([[18, 11, 15,  6,  1,  1]]), tensor([4]))
['<unk>', '<pad>', 'ー', 'ン', 'ル', 'ラ', 'ス', 'リ', 'ッ', 'ド', 'マ', 'イ', 'ク', 'シ', 'コ', 'ロ', 'ト', 'ア', 'カ', 'キ', 'バ', 'オ', 'タ', 'チ', 'グ', 'ジ', 'ブ', 'レ', 'ナ', 'ニ', 'フ', 'ム', 'ウ', 'ガ', 'ダ', 'ャ', 'ゴ', 'テ', 'プ', 'メ', 'デ', 'ノ', 'ュ', 'モ', 'ギ', 'ワ', 'ィ', 'パ', 'ボ', 'エ', 'ケ', 'ツ', 'ビ', 'サ', 'ネ', 'ハ', 'ヒ', 'ミ', 'ヤ', 'ゲ', 'ポ', 'ズ', 'ベ', 'ピ', 'ザ', 'ソ', 'ョ', 'ゼ', 'ヌ', 'ユ', 'ェ', 'ホ', 'セ', 'ペ', 'ヨ', 'ァ', 'ォ', 'ヘ', '・', '大', '小', 'A', 'N', '♀', '♂', 'ゾ', 'ヂ', '昼', '盾', '赤', '２', '：', 'Ｚ']


In [16]:
#モデルの定義
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torch.nn.functional as F

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class LSTMPVClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, lstm_hidden_size,  mlp_hidden_size, output_size):
        super(LSTMPVClassifier, self).__init__()
        self.lstm_hidden_size = lstm_hidden_size
   
        self.embed = nn.Embedding(vocab_size, embedding_dim, padding_idx=1)
        self.lstm = nn.LSTM(embedding_dim, self.lstm_hidden_size, batch_first=True, 
                        num_layers=1, bidirectional=False, dropout=0.0)
    
        self.fc1 = nn.Linear(self.lstm_hidden_size, mlp_hidden_size)
        self.fc2 = nn.Linear(mlp_hidden_size, output_size)
    
    def forward(self, x):
        b_size = x.size(0) # バッチサイズ
        seq_len  = x.size(1) # ポケモン名の長さ

        x = self.embed(x)
        h0 = torch.zeros(1, b_size, self.lstm_hidden_size).to(device)
        c0 = torch.zeros(1, b_size, self.lstm_hidden_size).to(device)
  
        lstm_output_seq, (h_n, c_n) = self.lstm(x, (h0, c0))
   
        out = torch.relu(self.fc1(lstm_output_seq))
        out = torch.sigmoid(self.fc2(out)) 

        return out

In [17]:
from sklearn.metrics import precision_recall_fscore_support as score
def scores(predicted, y_test):

    precision, recall, fscore, _ = score(y_test, predicted, average="binary")

    print('precision: {}'.format(precision))
    print('recall: {}'.format(recall))
    print('fscore: {}'.format(fscore))

def accuracy(pred, corr, p=0.5):
    acc = 0
    pred_label = []
    for i,pr in enumerate(pred):
        if pr < p:
            pred_label.append(0)
            if corr[i] == 0.0:
                acc += 1
            else:
                pred_label.append(1)
                if corr[i] == 1.0:
                    acc += 1
    return acc, pred_label

In [18]:
#ハイパラ設定
vocab_size = len(TEXT.vocab)
print('vocab size is {}'.format(str(vocab_size)))

lstm_hidden_size = 32
emb_dim = 16
mlp_hidden = 16
output_size = 1

net = LSTMPVClassifier(vocab_size, emb_dim, lstm_hidden_size, mlp_hidden, 
                        output_size)
net = net.to(device)

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, net.parameters()))

vocab size is 93


In [19]:
num_epochs = 10

train_loss_list = []
val_loss_list = []

for epoch in range(num_epochs):
    train_loss = 0.0
    val_loss = 0.0
    train_acc = 0.0
    val_acc = 0.0
    train_y = []
    train_predicted = []
    val_y = []
    val_predicted = []

    #train
    net.train()
    for i, batch in enumerate(train_iter):
        text = batch.name[0]
        text = text.to(device)
        if (text.size()[0] is not batch_size):
            continue
        labels = batch.bs
        labels = labels.to(device)
        f_labels = labels.float()
        optimizer.zero_grad()
        outputs = net(text)
        outputs = outputs[:, -1].squeeze()
#         print(f_labels.size())
#         print(outputs.size())
        loss = criterion(outputs, f_labels)
#         print(loss)
        train_loss += loss.item()
        tmp_acc, tmp_pred_label = accuracy(outputs, f_labels)
        train_acc += tmp_acc
        loss.backward()
        optimizer.step()

    avg_train_loss = train_loss / len(train_iter.dataset)
    avg_train_acc = train_acc / len(train_iter.dataset)

    #val
    net.eval()
    with torch.no_grad():
        for batch in val_iter:
            text = batch.name[0]
            text = text.to(device)
            if (text.size()[0] is not batch_size):
                continue
            labels = batch.bs
            labels = labels.to(device)
            f_labels = labels.float()
            outputs = net(text)
            outputs = outputs[:, -1].squeeze()
            loss = criterion(outputs, f_labels)
            val_loss += loss.item()
            tmp_acc, tmp_pred_label = accuracy(outputs, f_labels) 
            val_acc += tmp_acc
            val_predicted.extend(tmp_pred_label)
            val_y.extend(labels.tolist())


    avg_val_loss = val_loss / len(val_iter.dataset)
    avg_val_acc = val_acc / len(val_iter.dataset)
#     print ('Epoch [{}/{}], Loss: {loss:.4f}, Acc: {acc:.4f}, val_loss: {val_loss:.4f}, val_acc: {val_acc:.4f} \nP/R/F1 of val:' 
#                     .format(epoch+1, num_epochs, i+1, loss=avg_train_loss, acc=avg_train_acc, val_loss=avg_val_loss, val_acc=avg_val_acc))
    print ('Epoch [{}/{}], Loss: {loss:.4f}, Acc: {acc:.4f}' 
                    .format(epoch+1, num_epochs, i+1, loss=avg_train_loss, acc=avg_train_acc))
#     scores(val_predicted, val_y)

    torch.save(net.state_dict(), "./model/model_cate_{0:03d}.pwf".format(epoch+1)) 

Epoch [1/10], Loss: 0.1712, Acc: 0.9248
Epoch [2/10], Loss: 0.1639, Acc: 0.5972
Epoch [3/10], Loss: 0.1512, Acc: 0.5580
Epoch [4/10], Loss: 0.1404, Acc: 0.5596
Epoch [5/10], Loss: 0.1294, Acc: 0.5392
Epoch [6/10], Loss: 0.1225, Acc: 0.5172
Epoch [7/10], Loss: 0.1170, Acc: 0.5204
Epoch [8/10], Loss: 0.1092, Acc: 0.5172
Epoch [9/10], Loss: 0.1013, Acc: 0.5188
Epoch [10/10], Loss: 0.0941, Acc: 0.5219


In [20]:
from tqdm import tqdm
import time

best_epoch = 10
vocab_size = len(TEXT.vocab)
eval_net = LSTMPVClassifier(vocab_size, emb_dim, lstm_hidden_size, mlp_hidden,  output_size)
eval_net = eval_net.to(device)
load_model = "./model/model_cate_{0:03d}.pwf".format(best_epoch)
eval_net.load_state_dict(torch.load(load_model))
test_acc = 0.0
test_predicted = []
test_y = []

# 合計値(total)を設定
# bar = tqdm(total = len(val_iter))

def labeling(pred, p=0.5):

    pred_label = []
    for i,pr in enumerate(pred):
        if pr < p:
            pred_label.append(0)
        else:
            pred_label.append(1)
    
    return pred_label


eval_net.eval()
with torch.no_grad():
  #ここをtestから、全部に書き換えて特徴量にしたい
    for batch in val_iter:
        text = batch.name[0]
        text = text.to(device)
        outputs = eval_net(text)
        outputs = outputs[:, -1]
        # tmp_pred_label = labeling(outputs) 
        tmp_pred_label = outputs
        test_predicted.extend(tmp_pred_label)

#         bar.update(1)

# with open('./train.txt', 'w') as g:
#     g.write('answer' +'\n')
#     for line in test_predicted:
#         g.write(str(line) + '\n')
# print('inference done...')

In [21]:
labeled_predicted = labeling(test_predicted)
labeled_predicted

[1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0]

In [22]:
val_df = pd.read_csv('val_df.tsv', delimiter='\t')
val_df['予測値'] = labeled_predicted
val_df

,Unnamed: 0,ポケモン名,合計,予測値
0,154,カ イ ロ ス,1,1
1,240,グ ラ イ ガ ー,0,1
2,371,バ ク ー ダ,1,0
3,408,ト ロ ピ ウ ス,1,1
4,667,エ モ ン ガ,0,0
...,...,...,...,...
155,636,マ ラ カ ッ チ,1,0
156,121,ス リ ー パ ー,1,0
157,559,パ ル キ ア,1,0
158,55,ナ ゾ ノ ク サ,0,0


In [23]:
pd.set_option('display.max_rows', 200)
val_df

,Unnamed: 0,ポケモン名,合計,予測値
0,154,カ イ ロ ス,1,1
1,240,グ ラ イ ガ ー,0,1
2,371,バ ク ー ダ,1,0
3,408,ト ロ ピ ウ ス,1,1
4,667,エ モ ン ガ,0,0
5,157,コ イ キ ン グ,0,1
6,204,ピ チ ュ ー,0,0
7,358,プ ラ ス ル,0,0
8,53,ズ バ ッ ト,0,0
9,481,ビ ー ク イ ン,1,1


In [24]:
score = scores(val_df['合計'], val_df['予測値'])
score

precision: 0.6338028169014085
recall: 0.6521739130434783
fscore: 0.6428571428571428


### 二値分類における評価指標.

二値分類の評価


|----|予測値(正解)|予測値(不正解)|
|----|:-|:-|
|正解値(正解)|TP : True Positive|FN : False Negative|
|正解値(不正解)|FP : False Positive|TN : True Negative|

##### 適合率（Precision）

$$
    \frac{TP}{TP+FP}
$$

適合率は, 精度評価でFP（偽陽性＝不正解＆陽性予測）をできるだけ低くしたい.  
FN（偽陰性＝不正解＆陰性予測）があまり重要ではなく無視してもよい場合に有用.


##### 再現率（Recall）

$$
    \frac{TP}{TP+FN}
$$

適合率の逆.

##### F値 (F-score)

$$
\frac{2Recall×Precision}{Recall+Precision}
$$

適合率と再現率の調和平均.   

<br>

**¶とりあえずモデル完成**

## 最強のポケモン生成

#### 論文の手法
- 適当にサンプルを選定
- ランダムに一文字置き換える
- 置き換えた名前とモデルを用いて比較
- 上記3工程を50回ループ

同じ手法で生成していく.

### 適当な名前入れて数字の確認

In [38]:
#データセット化
#変換後の数値をチェック
def to_dataset(list_obj, pri=True):
    index = pd.DataFrame(list_obj)
    index[0] = index[0].map(function)
    index.to_csv('./test.tsv', sep='\t')
    
    fields_test = [('id', None), ('name', TEXT)]
    dataset_test = data.TabularDataset(path='./test.tsv',
                             format='TSV', skip_header=True, fields=fields_test)
    
    test_iter = data.BucketIterator(dataset=dataset_test, batch_size=1, 
                                 sort_key=lambda x: len(x.name), repeat=False, shuffle=False)
    
    batch = next(iter(test_iter))
    
    if pri:
        print(batch.name)
    
    return test_iter

list_obj = ['デンシケトル', 'ガギグゲゴ', 'デンシレンジ', 'フライパン', 'ジサボケ', 'ポケモン']
test_iter = to_dataset(list_obj)

(tensor([[40,  3, 13, 50, 16,  4]]), tensor([6]))


In [39]:
def result_show(test_iter, pri=True):
    test_predicted = []
    
    for batch in test_iter:
        text = batch.name[0]
        text = text.to(device)
        outputs = eval_net(text)
        outputs = outputs[:, -1]
        tmp_pred_label = outputs.to('cpu').detach().numpy().copy()
        test_predicted.extend(tmp_pred_label[0])
    
    if pri:
        print(test_predicted)
    
    return test_predicted
        
result = result_show(test_iter)

[0.9198542, 0.71148044, 0.9522135, 0.79839903, 0.1434933, 0.05419633]


In [40]:
df = pd.DataFrame(list_obj, columns=['名前'])
df['予測値'] = result
df['0,1ラベル'] = labeling(df['予測値'])

In [41]:
df

,名前,予測値,"0,1ラベル"
0,デンシケトル,0.919854,1
1,ガギグゲゴ,0.711480,1
2,デンシレンジ,0.952214,1
3,フライパン,0.798399,1
4,ジサボケ,0.143493,0
5,ポケモン,0.054196,0


### 名前を入れ替えて最強のポケモン生成！

生成手順(再掲)
- 適当にサンプルを選定
- ランダムに一文字置き換える
- 置き換えた名前とモデルを用いて比較
- 上記3工程を50回ループ

<br>
論文では"パラセクト", "ニドクイン"から生成だったのでそれに倣う.

名前を交換するコードから.

In [29]:
import random
letters = 'アイウエオカキクケコサシスセソタチツテトナニヌネノハヒフヘホマミムメモヤユヨラリルレロワヲンドガグバジブゴダプャュデパボギゲピポズベョザゼペゾー'

def change_name(string, counter):
    for i in range(counter):
        choiced = random.choice(letters)
        ran = random.randrange(len(string))
        string = string.replace(string[ran], random.choice(choiced))
    return string

string = 'パラセクト'
string = change_name(string, 1)
string

'パラベクト'

### 次に, スコアの比較機能を導入する

In [30]:
string = 'パラセクト'
changed_string = change_name(string, 1)
cd_result = result_show(to_dataset([string, changed_string]))

cd_df = pd.DataFrame([string, changed_string], columns=['名前'])
cd_df['予測値'] = cd_result
cd_df

(tensor([[47,  5, 72, 12, 16,  1]]), tensor([5]))
[0.103751846, 0.32669568]


,名前,予測値
0,パラセクト,0.103752
1,パラネクト,0.326696


In [44]:
def generate_pokemon(string):
    history = []
    score = []
    history.append(string)
    
    for i in range(50):
        changed_string = change_name(string, 1)
        cd_result = result_show(to_dataset([string, changed_string], False), False)
        
        #最初だけ追加
        if i ==0:
            score.append(cd_result[0])
            
        if cd_result[0] > cd_result[1]:
            score.append(cd_result[0])
        else:
            string = changed_string
            score.append(cd_result[1])
            
        history.append(string)
        
    cd_df = pd.DataFrame(history, columns=['名前'])
    cd_df['予測値'] = score
    
    return string, cd_df


string = 'パラセクト'
saikyou, port = generate_pokemon(string)

print('最強の名前 : ', saikyou)
pd.DataFrame(port)

最強の名前 :  エギネオオ


,名前,予測値
0,パラセクト,0.103752
1,パラセトト,0.107857
2,パラセアア,0.321263
3,パラセアア,0.321263
4,パギセアア,0.696650
5,パギセアア,0.696650
6,パギセアア,0.696650
7,シギセアア,0.925912
8,シギセアア,0.925912
9,ムギセアア,0.943413


In [45]:
string = 'ニドクイン'
saikyou, port = generate_pokemon(string)

print('最強の名前 : ', saikyou)
pd.DataFrame(port)

最強の名前 :  ペギョョュ


,名前,予測値
0,ニドクイン,0.761270
1,ニドボイン,0.863098
2,ニドボイン,0.863098
3,ニドジイン,0.900434
4,ニドジョン,0.917604
5,ニドジョン,0.917604
6,ニドジョン,0.917604
7,ニドジョン,0.917604
8,ニドジョン,0.917604
9,ニドジョン,0.917604


**以上が生成結果**

<br><br><br><br><br><br><br>

### 実は若干問題のある確認手法

<br><br><br><br>

**trainデータとvalデータの区別がついていない**

In [33]:
val_df

,Unnamed: 0,ポケモン名,合計,予測値
0,154,カ イ ロ ス,1,1
1,240,グ ラ イ ガ ー,0,1
2,371,バ ク ー ダ,1,0
3,408,ト ロ ピ ウ ス,1,1
4,667,エ モ ン ガ,0,0
5,157,コ イ キ ン グ,0,1
6,204,ピ チ ュ ー,0,0
7,358,プ ラ ス ル,0,0
8,53,ズ バ ッ ト,0,0
9,481,ビ ー ク イ ン,1,1


In [34]:
string = 'デルビル'
saikyou, port = generate_pokemon(string)

print('最強の名前 : ', saikyou)
pd.DataFrame(port)

最強の名前 :  ヲレムレ


,名前,予測値
0,デルビル,0.042866
1,デルビル,0.042866
2,デョビョ,0.049263
3,ムョビョ,0.371044
4,ムョビョ,0.371044
5,ロョビョ,0.525229
6,ロョビョ,0.525229
7,ロンビン,0.740337
8,ロンビン,0.740337
9,ロンビン,0.740337


## 仮説
##### 結果が同じになっているがわからないが, 何度か試したところ, 音情報と同じく濁点半濁点付きの名前が強い傾向にありそう.

だったら検証してみよう！

In [35]:
dakuten_list  = 'ガギグゲゴザジズゼゾダヂヅデドバビブベボパピプペポ'
def count_dakuten(name):
    counter = 0
    for n in name:
        if n in dakuten_list:
            counter += 1
            
    return counter 

status['sum_濁点'] = status['ポケモン名'].map(count_dakuten)
status

,ポケモン名,合計,sum_濁点
0,フ シ ギ ダ ネ,0,2
1,フ シ ギ ソ ウ,0,1
2,フ シ ギ バ ナ,1,2
4,ヒ ト カ ゲ,0,1
5,リ ザ ー ド,0,2
...,...,...,...
904,カ ミ ツ ル ギ,1,1
905,ア ク ジ キ ン グ,1,2
906,ネ ク ロ ズ マ,1,1
907,マ ギ ア ナ,1,1


### 既存のポケモンの強さと濁点(半濁点)の量の相関係数を取ってみる

In [36]:
cr = status.index
cr_matrix = status.corr()
print(cr_matrix)

             合計   sum_濁点
合計      1.00000  0.10872
sum_濁点  0.10872  1.00000


#### ほぼほぼ相関なし

もしかしたら, そもそも名前に濁点が含まれているのが多い?

In [37]:
print(status['sum_濁点'].value_counts())

1    335
0    270
2    174
3     18
4      1
Name: sum_濁点, dtype: int64


#### 結果

多いといえば多い結果.   
濁点の数が二つを超えるケースが多々あったので, そことの相関を考えると意外とありそう？

## まとめ

**全体的におもしろい結果になったと思う.**

<br>

今回ポケモンだけど, 例えばラーメン店の名前と, 食べログ評価とかでやっても面白そう.  
深層学習のモデルが若干大掛かりなもの使ってるから, その他手法でもいろいろ試せそう.  

<br>

参考資料
>https://qiita.com/taku910/items/7e52f1e58d0ea6e7859c

>https://ishitonton.hatenablog.com/entry/2018/11/25/200332  

>Vo, Quan-Hoang & Nguyen, Huy-Tien & Le, Bac & Nguyen, Minh-Le. (2017). Multi-channel LSTM-CNN model for Vietnamese sentiment analysis. 24-29. 10.1109/KSE.2017.8119429. 